# ハイパーパラメータのチューニングを行った結果を可視化する

#### ノートブックに含まれる内容

- ハイパーパラメータの結果を可視化するサンプル
- boto3 経由での SageMaker API アクセスと，pandas + bokeh による可視化のやり方

## セットアップ

以下の `tuning_job_name` を，**<span style="color: red;">先ほど実行した自分のチューニングジョブ名に変更</span>**してください．マネジメントコンソールから，チューニングジョブの一覧を確認することが可能です．

In [ ]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

tuning_job_name = 'sagemaker-tensorflow-XX-XXXXX-XXXX'

## チューニングジョブの進行状況を確認して結果を取得

ハイパーパラメータチューニングのジョブ進行状況を確認して，終わっていたら結果を取り出します．

In [ ]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

pprint でジョブの結果を出力してみます．

In [ ]:
from pprint import pprint
print("Best model found so far:")
pprint(tuning_job_result['BestTrainingJob'])

## 全てのジョブの結果を取り出して DataFrae に格納

各ジョブについて，ジョブ概要に加えて，ハイパーパラメータの値とターゲットメトリクスを取り出して，表にまとめます．詳細については[ドキュメント](https://sagemaker.readthedocs.io/en/latest/analytics.html?highlight=HyperparameterTuningJobAnalytics)を参照してください．

In [ ]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

##  Bokeh による結果の可視化

### チューニングジョブの進展によるターゲットメトリクスの改善

続いて，学習ジョブの進行によるターゲットメトリクスの改善についてみていきます．チューニングジョブは裏側でベイズ最適化を行なっており，見探索の空間に対する探索と，すでに得た情報の活用とのバランスをとりながら，最適な形でパラメータ探索を行なっていきます．

In [ ]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900,
           plot_height=400,
           tools=hover.tools(),
           x_axis_type='datetime',
           x_axis_label='training start time(min)', y_axis_label=objective_name,)
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

### 特定のハイパーパラメータとターゲットメトリクスの関連を可視化

続いて，複数あるハイパーパラメータについて，特定のひとつの値を変化させたときに，どのくらいターゲットメトリクスが改善するかを可視化します．これにより，最適なハイパーパラメータがどのあたりにあるかを，個々のパラメータごとに確認することができます．

In [ ]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))